In [24]:
import pandas as pd
import plotly.express as px
from google.colab import files
from IPython.display import display, HTML


def read_the_csv(file, year):
    df = pd.read_csv(file, sep=",", engine="python")
    df["Year"] = year
    return df

df_2024 = read_the_csv("https://raw.githubusercontent.com/pjfernandes/university_ranking/refs/heads/main/the_2024_full.csv", 2024)
df_2025 = read_the_csv("https://raw.githubusercontent.com/pjfernandes/university_ranking/refs/heads/main/the_2025_full.csv", 2025)
df_2026 = read_the_csv("https://raw.githubusercontent.com/pjfernandes/university_ranking/refs/heads/main/the_2026_full.csv", 2026)

df = pd.concat([df_2024, df_2025, df_2026], ignore_index=True)

print("\nColumns detected:")
print(df.columns.tolist())

df.columns = df.columns.str.strip()

df_brazil = df[df["Country"].str.contains("Brazil", case=False, na=False)].copy()


numeric_cols = [
        #"Overall",
        "Teaching",
        "Research Environment",
        "Research Quality",
        "Industry",
        "International Outlook",
    ]

for col in numeric_cols:
        if col in df_brazil.columns:
            df_brazil[col] = (
                df_brazil[col]
                .astype(str)
                .str.replace("–", "-")
                .str.extract(r"([\d.]+)")
                .astype(float)
            )


uff_name = "Fluminense Federal University"
compare_to = "Federal University of São Carlos"

df_filtered = df_brazil[df_brazil["Name"].isin([uff_name, compare_to])]

indicators = numeric_cols

from plotly.subplots import make_subplots
import plotly.express as px
from IPython.display import display, HTML

display(HTML(f"<h2>Comparison: {uff_name} vs. {compare_to}</h2>"))

# Número de gráficos
n = len(indicators)
cols = 2
rows = (n + 1) // 2  # arredonda para cima

# Cria o layout de subplots
fig_combined = make_subplots(
    rows=rows,
    cols=cols,
    subplot_titles=indicators
)

# Contador de posição
row = 1
col = 1
first = True  # para exibir a legenda apenas uma vez

for metric in indicators:
    if metric in df_filtered.columns:
        fig = px.line(
            df_filtered,
            x="Year",
            y=metric,
            color="Name",
            markers=True,
            line_dash="Name",
            color_discrete_map={uff_name: "blue", compare_to: "red"},
        )
        for trace in fig.data:
            # Legenda apenas para a primeira figura
            trace.showlegend = first
            fig_combined.add_trace(trace, row=row, col=col)

        first = False  # desativa legenda

        # Atualiza posição
        if col == 1:
            col = 2
        else:
            col = 1
            row += 1

fig_combined.update_layout(
    height=300*rows,
    width=1500,
    template="plotly_white",
    legend_title_text="University",
    title_text="Indicadores (THE Ranking)",
)

fig_combined.show()


display(HTML("<h3>All Brazilian universities (sample)</h3>"))
display(df_brazil.head(10))



Columns detected:
['Rank', 'Name', 'Country', 'Overall', 'Teaching', 'Research Environment', 'Research Quality', 'Industry', 'International Outlook', 'Year']


,Rank,Name,Country,Overall,Teaching,Research Environment,Research Quality,Industry,International Outlook,Year
0,201–250,University of São Paulo,Brazil,55.9–58.6,59.8,60.7,57.3,68.2,42.5,2024
1,351–400,University of Campinas,Brazil,49.1–51.0,50.0,45.5,54.0,65.5,40.4,2024
2,601–800,Universidade Estadual Paulista (Unesp),Brazil,37.0–41.8,41.5,33.1,38.1,40.5,38.6,2024
3,601–800,Federal University of Rio de Janeiro,Brazil,37.0–41.8,41.8,36.0,36.7,81.0,35.9,2024
4,601–800,Universidade Federal do Rio Grande do Sul,Brazil,37.0–41.8,36.4,22.5,54.1,52.9,34.1,2024
5,801–1000,Federal University of Minas Gerais,Brazil,32.7–36.9,40.2,23.3,43.3,50.8,33.4,2024
6,801–1000,Universidade Federal de Santa Catarina,Brazil,32.7–36.9,32.5,21.6,41.6,64.2,35.3,2024
7,801–1000,Universidade Federal de São Paulo (UNIFESP),Brazil,32.7–36.9,33.2,22.5,48.2,47.2,35.2,2024
8,801–1000,Pontifical Catholic University of Rio de Janei...,Brazil,32.7–36.9,33.4,30.5,37.1,62.9,43.2,2024
9,1001–1200,Federal University of São Carlos,Brazil,28.3–32.6,33.4,19.8,33.5,44.8,29.5,2024
